Problem 3 - Genome Assembly as Shortest Superstring

In [61]:
import re
import os

input = "R3_testSC.fasta" #Input file

##0.Eliminate \n jumps to have one strig for each record
def fasta_no_jump(imput, out):
    fasta_file = open(imput, "rt")
    output = open(out, "wt")

    lin = fasta_file.readline()
    output.write(lin)

    for line in fasta_file:
        match = re.search(r"^>",line)
        if match:
            output.write("\n" + line)
        else:
            output.write(line.replace("\n",""))

    fasta_file.close()
    output.close()

fasta_no_jump(input, "./NoJump.fasta")
input1 = open("./NoJump.fasta", "rt")

##1.Append the tags and the sequences of the input(NoJump) file into two lists
tag_lst = []
seq_lst = []

while True: #for each record
    tag = input1.readline() #Tag
    if not tag: #If there is no tag, break the loop
        break
    seq = input1.readline() #Sequence
    
    tag_lst.append(tag[1:len(tag)].replace("\n","")) #Append tag without > and \n 
    seq_lst.append(seq.replace("\n","")) #Append sequence without \n

input1.close() #close NoJump file
os.remove("./NoJump.fasta") #Remove NoJump file



##2.Create a file with the overlap graph

#Calculate the max overlapping bp between 2 sequences
def max_overlap(seq1, seq2):
    
    min_l = min(len(seq1), len(seq2))

    for k in range(0,min_l+1):
        if seq1[len(seq1)-k:len(seq1)] == seq2[0:k]:
            overlap_bp = k
    
    return overlap_bp

#Calculate the overlapping graph as an Array in the way (seq1 seq2 overlap bp)

def overlap_graph_generator(seql, tagl):
    overlap_graph_list = []

    for x in range(0,len(seql)): #For each pair of sequences in the list (s1 and s2)
        for y in range(0,len(seql)): 
            s1 = seql[x]
            s2 = seql[y]
            if s1 != s2: #If they are not the same sequence...
                overlap_bp = max_overlap(s1,s2)
                tag1 = tagl[x] #tag of s1 is tag1
                tag2 = tagl[y] #tag of s2 is tag2

                new_entry = [tag1, tag2, overlap_bp]
                overlap_graph_list.append(new_entry) #Writte tail and head tags (v,w) in output file
    
    return overlap_graph_list

###

def merge_most_overlap(sequence_list, tag_list, ov_graph):
    overlap_val = []

    for record in ov_graph:
        overlap_val.append(record[2])
    index = overlap_val.index(max(overlap_val))

    overlap_record = ov_graph[index]
    s1 = sequence_list[tag_list.index(overlap_record[0])]
    s2 = sequence_list[tag_list.index(overlap_record[1])]
    n = overlap_record[2]
    
    new_sequence = s1[0:len(s1)-n] + s2 #Merge two overlap sequences
    return new_sequence, overlap_record

#Recursive function
def recursive_assembly(seq_lst, tag_lst, count = 0):
    
    if len(seq_lst) == 1:
        assembly = seq_lst[0]
        return assembly
    
    else:
        overlap_graph = overlap_graph_generator(seq_lst, tag_lst)
        new_seq, record = merge_most_overlap(seq_lst, tag_lst, overlap_graph)

        count += 1 
        new_tag = "MergedSeq_" + str(count)

        seq_lst.pop(tag_lst.index(record[0]))
        tag_lst.pop(tag_lst.index(record[0]))

        seq_lst.pop(tag_lst.index(record[1]))
        tag_lst.pop(tag_lst.index(record[1]))
        
        seq_lst.append(new_seq) #calculate new seq (remove s1 and s2, append colapse)
        tag_lst.append(new_tag) #calculate mew tag (remove tag1 and tag2, append colapse tag)
    
        assembly = recursive_assembly(seq_lst, tag_lst, count)
    
    return assembly

shortest_superstring = recursive_assembly(seq_lst, tag_lst)

out = open("./R3_output.txt", "wt")
out.write(shortest_superstring)
out.close()

#Falta:
#Revisar y ponerlo el código bonito
#Comentarlo